In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display

# 지역 이름과 코드 매핑
regions = {
    "남구": "5",
    "부산진구": "8",
    "해운대구": "18",
    "동래구": "7",
    "사하구": "12",
    "북구": "10",
    "사상구": "11"
}

# 날짜 설정
target_date = "2016-04-07"

# 데이터 수집 함수
def fetch_region_data(region_name, region_code):
    url = "https://www.busan.go.kr/depart/abnecessity01"
    params = {
        "isMart": "true",
        "kind": "",
        "schDate": target_date,
        "schCode": "0",
        "schLocale": region_code,
        "schMart": "0"
    }

    response = requests.get(url, params=params)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find("table", class_="boardList")

    region_data = []

    if table:
        rows = table.find_all("tr")
        for row in rows:
            tds = row.find_all("td")
            td_texts = [td.get_text(strip=True) for td in tds]
            if len(td_texts) >= 5:
                item_name = td_texts[0]
                spec = td_texts[1]
                try:
                    lowest_price = int(td_texts[2].replace(",", ""))
                    highest_price = int(td_texts[3].replace(",", ""))
                except ValueError:
                    continue
                avg_price = td_texts[4]
                region_data.append([item_name, spec, lowest_price, highest_price, avg_price, region_name])
    return region_data

# 모든 지역에서 데이터 수집
all_data = []
for region_name, region_code in regions.items():
    all_data.extend(fetch_region_data(region_name, region_code))

# DataFrame 생성
columns = ["품목", "규격", "최저가", "최고가", "금주평균가", "지역"]
df = pd.DataFrame(all_data, columns=columns)

# 최저가, 최고가 각각 추출
min_df = df.loc[df.groupby("품목")["최저가"].idxmin()][["품목", "규격", "최저가", "지역", "금주평균가"]].reset_index(drop=True)
max_df = df.loc[df.groupby("품목")["최고가"].idxmax()][["품목", "최고가", "지역"]].reset_index(drop=True)

# 컬럼명 변경
min_df = min_df.rename(columns={"지역": "최저가_지역"})
max_df = max_df.rename(columns={"지역": "최고가_지역"})

# 병합
merged_df = pd.merge(min_df, max_df, on="품목")

# 가격 차이 계산
merged_df["가격차이"] = merged_df["최고가"] - merged_df["최저가"]
merged_df["차이비율(%)"] = (merged_df["가격차이"] / merged_df["최저가"] * 100).round(2).astype(str) + "%"

# 날짜 열 추가
merged_df["날짜"] = target_date

# 열 정리 및 출력
merged_df = merged_df[["날짜", "품목", "규격", "최저가", "최저가_지역", "최고가", "최고가_지역", "가격차이", "차이비율(%)", "금주평균가"]]

# 표 출력
display(merged_df)

# 저장하고 싶다면
merged_df.to_csv("2016_total_2q.csv", index=False, encoding="utf-8-sig")


,날짜,품목,규격,최저가,최저가_지역,최고가,최고가_지역,가격차이,차이비율(%),금주평균가
0,2016-04-07,가루비누,"세탁용세제(LG테크),가루형,3.0kg",5400,북구,17850,남구,12450,230.56%,"6,038"
1,2016-04-07,간장,오복왕표 0.9ℓ 1병,3754,해운대구,7360,사상구,3606,96.06%,"4,985"
2,2016-04-07,고등어,30㎝정도 500g 1마리,2405,사상구,6980,남구,4575,190.23%,"4,329"
3,2016-04-07,달걀,오경슈퍼란60g정도 10개,1960,해운대구,3000,동래구,1040,53.06%,"2,413"
4,2016-04-07,닭고기,육계 1.0㎏,7,동래구,12900,부산진구,12893,184185.71%,"3,004"
5,2016-04-07,대파,1.0㎏,2592,해운대구,5653,동래구,3061,118.09%,"4,119"
6,2016-04-07,돼지고기,삼겹살 500g,6400,동래구,12950,부산진구,6550,102.34%,"8,000"
7,2016-04-07,두부,500g 판두부(국산포장두부 420g) 1모,3250,북구,5300,남구,2050,63.08%,"3,808"
8,2016-04-07,라면,신라면 120g 1봉지,630,남구,634,동래구,4,0.63%,630
9,2016-04-07,맥주,하이트 500㎖ 1병,1240,남구,1350,해운대구,110,8.87%,"1,240"
